In [1]:
import sys, os
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("Application"). \
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        config("spark.hadoop.fs.s3a.access.key", f"{os.environ['MINIO_ROOT_USER']}").\
        config("spark.hadoop.fs.s3a.secret.key", f"{os.environ['MINIO_ROOT_PASSWORD']}").\
        config("spark.hadoop.fs.s3a.endpoint", f"http://{os.environ['MINIO_IP']}:9000").\
        config("spark.eventLog.enabled", "true").\
        config("spark.eventLog.dir", "s3a://spark-events/").\
        config("spark.history.fs.logDirectory", "s3a://spark-events/").\
        getOrCreate()


23/12/18 07:36:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/local/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [2]:
import sys, os
os.environ['MINIO_IP']

'172.20.0.2'

In [3]:
conf = spark.sparkContext.getConf().getAll()

# Print Configuration
for k,v in conf:
    print(f"{k} : {v}")

spark.eventLog.enabled : true
spark.executor.memory : 512m
spark.hadoop.fs.s3a.connection.ssl.enabled : false
spark.driver.extraJavaOptions : -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED
spark.eventLog.dir : s3a://spark-events/
spark.app.name : Application
spark.hadoop.fs.s3a.path.style.access : true
spark.hadoop.fs.s

In [4]:
df = spark.read.csv(f"s3a://{os.environ['MINIO_BUCKET']}/yellow_tripdata_2021-01.csv", header=True)

In [5]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2021-01-01 00:30:10|  2021-01-01 00:36:12|              1|         2.10|         1|                 N|         142|          43|           2|          8|    3|    0.5|         0|           0|                  0.3

In [6]:
df.write.parquet(f"s3a://{os.environ['MINIO_BUCKET']}/yellow_tripdata/2021/01/", mode='overwrite')

In [7]:
df_parquet = spark.read.parquet(f"s3a://{os.environ['MINIO_BUCKET']}/yellow_tripdata/2021/01/")

In [8]:
df_parquet.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2021-01-01 00:30:10|  2021-01-01 00:36:12|              1|         2.10|         1|                 N|         142|          43|           2|          8|    3|    0.5|         0|           0|                  0.3

In [9]:
spark.stop()